## Create Dataset

### 1. Label images

Use labelme or something to label the image set you've got. Images should be labelled with different classes and each class has a unique id.

In [ ]:
classdict = {
    "liner": 0,
    "bulk carrier": 1,
    "warship": 2,
    "sailboat": 3,
    "canoe": 4,
    "container ship": 5,
    "fishing boat": 6
} # How I set Infrared-Ocean-Target images' classes

### 2. Orgnize structure

After labeling the images, your dataset structure should look like this:

```
RawDataset/
├── annotations/
│   ├── 1_1.xml
│   └── 1_3.xml
└── images/
    ├── 1_1.jpg
    └── 1_3.jpg
```

In [ ]:
rawdataset_dir = './RawDataset' # rawdataset rootdir
labels_dir = f'{rawdataset_dir}/annotations' # rootdir/xmldir
images_dir = f'{rawdataset_dir}/images' # rootdir/jpgdir

## Preprocess Dataset

According to the ultralytic's offical document, our final dataset structure should look like this:

```
Dataset/
├── train/
│   ├── images/
│   │   └── 1_1.txt
│   └── labels/
│       └── 1_1.jpg
└── val/
    ├── images/
    │   └── 1_3.txt
    └── labels/
        └── 1_3.jpg
```

In [ ]:
dataset_dir = './Dataset' # dataset rootdir
dataset_dir_train = f'{dataset_dir}/train' # rootdir/traindir
dataset_dir_val = f'{dataset_dir}/val' # rootdir/valdir

### 1. Split dataset

To divide dataset, we usually first shuffle the files and then divide them by ratio of 7:3 (7 for train anf 3 for validation).

In [ ]:
import os
import random
from pathlib import Path


xmlfiles = [Path(labels_dir).joinpath(f).as_posix() for f in os.listdir(labels_dir) if f.endswith('.xml')]
#imagefiles = [Path(images_dir).joinpath(f).as_posix() for f in os.listdir(images_dir) if f.endswith('.jpg')]

random.shuffle(xmlfiles)

TrainSize = int(len(xmlfiles) * 0.7)
xmlFiles_Train = xmlfiles[:TrainSize]
xmlFiles_Val = xmlfiles[TrainSize:]

jpgFiles_Train = [Path(images_dir).joinpath(os.path.basename(f).replace('xml', 'jpg')).as_posix() for f in xmlFiles_Train]
jpgFiles_Val = [Path(images_dir).joinpath(os.path.basename(f).replace('xml', 'jpg')).as_posix() for f in xmlFiles_Val]

### 2. Convert xml to txt

So the next step is to convert xml to txt with the following content format that yolo supported:

```
classid center_x center_y width height
```

In [ ]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path


def convert_xml_to_yolotxt(classdict, xml_path, output_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    objects = root.findall('object')
    size = root.find('size')
    size_w = int(size.find('width').text)
    size_h = int(size.find('height').text)

    with open(output_path, 'w') as f:
        def calc(box, size_w, size_h):
            x = (box[0] + box[2]) / 2.0
            y = (box[1] + box[3]) / 2.0
            w = box[2] - box[0]
            h = box[3] - box[1]
            x = x / size_w
            y = y / size_h
            w = w / size_w
            h = h / size_h
            return x, y, w, h
        for obj in objects:
            classid = obj.find('name').text
            classid = classdict[classid]
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            box = (xmin, ymin, xmax, ymax)
            center_x, center_y, width, height = calc(box, size_w, size_h)
            f.write(f"{classid} {center_x} {center_y} {width} {height}\n")


for xmlfile in xmlFiles_Train:
    labelsoutput_dir_train = Path(dataset_dir_train).joinpath('labels').as_posix()
    os.makedirs(labelsoutput_dir_train, exist_ok = True)
    convert_xml_to_yolotxt(
        classdict,
        xmlfile,
        Path(labelsoutput_dir_train).joinpath(os.path.basename(xmlfile).split('.')[0] + '.txt').as_posix()
    )

for xmlfile in xmlFiles_Val:
    labelsoutput_dir_val = Path(dataset_dir_val).joinpath('labels').as_posix()
    os.makedirs(labelsoutput_dir_val, exist_ok = True)
    convert_xml_to_yolotxt(
        classdict,
        xmlfile,
        Path(labelsoutput_dir_val).joinpath(os.path.basename(xmlfile).split('.')[0] + '.txt').as_posix()
    )

### 3. Move images

EZ :)

In [ ]:
import os
import shutil
from pathlib import Path


for jpgfile in jpgFiles_Train:
    imagesoutput_dir_train = Path(dataset_dir_train).joinpath('images').as_posix()
    os.makedirs(imagesoutput_dir_train, exist_ok = True)
    shutil.move(jpgfile, imagesoutput_dir_train)

for jpgfile in jpgFiles_Val:
    imagesoutput_dir_val = Path(dataset_dir_val).joinpath('images').as_posix()
    os.makedirs(imagesoutput_dir_val, exist_ok = True)
    shutil.move(jpgfile, imagesoutput_dir_val)

## Create YAML

Before we start training, we still need a yaml file to help yolo load dataset correctly.

In [ ]:
# Dataset - Paths
path = 'D:/YOLOv8-Basic-Tutorial/Dataset' # [essential] dataset root dir (better use absolute path to avoid error)
train = 'train/images'  # [essential] train images (relative to 'path')
val = 'val/images'  # [essential] val images (relative to 'path')

# Dataset - Classes (Example)
class0 = 'liner'
class1 = 'bulk carrier'
class2 = 'warship'
class3 = 'sailboat'
class4 = 'canoe'
class5 = 'container ship'
class6 = 'fishing boat'

Here we use python's yaml lib to create file. Note that the file should be located in your project's root dir (The same dir with training file).

In [ ]:
import yaml


yamldata = f'''
# Note: This yaml file is created with python

# Paths
path: {path}
train: {train}
val: {val}

# Classes (Example)
names:
  0: {class0}
  1: {class1}
  2: {class2}
  3: {class3}
  4: {class4}
  5: {class5}
  6: {class6}

# Done
'''

with open('./Infrared-Ocean-Target.yaml', 'w', encoding = 'utf-8') as file:
    yaml.dump(
        yaml.safe_load(yamldata),
        file,
        default_flow_style = False,
        allow_unicode = True
    )